In [1]:
import os

import pandas as pd
import numpy as np

#from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer#, IterativeImputer
from sklearn.feature_selection import SelectKBest, mutual_info_regression, f_regression## exp needed, r_regression!
from sklearn.ensemble import IsolationForest## exp needed!

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.svm import SVR

from sklearn.metrics import r2_score

In [2]:
root_path = './'
X_train_path = 'X_train.csv'
X_test_path = 'X_test.csv'
y_train_path = 'y_train.csv'
y_test_path = 'y_test_yutong_v2.csv'

num_features = 25
val_size = 101
random_state = 29

In [3]:
def data_raw(root_path, data_path):
    return pd.read_csv(os.path.join(root_path, data_path)).values[:,1:]

def median_imp(X_raw):
    imp_med = SimpleImputer(missing_values=np.nan, strategy='median')
    return imp_med.fit_transform(X_raw)

def mice_imp(X_raw):#use after feature sel! this is comp expensive
    imp_mice = IterativeImputer(missing_values=np.nan, initial_strategy='median')
    return imp_mice.fit_transform(X_raw)

def feat_sel(X_raw, y, score_func, num_features):
    return SelectKBest(score_func = score_func, k=num_features).fit(X_raw, y.ravel())

def train_val_split(X_train, y_train, val_size, random_state):
    return train_test_split(X_train, y_train, test_size=val_size, random_state=random_state)

def out_det(X_raw, y, random_state):
    iso_for = IsolationForest(random_state=random_state).fit_predict(X_raw)
    return X_raw[np.where(iso_for==1)], y[np.where(iso_for==1)]

In [4]:
X_train_raw = data_raw(root_path, X_train_path)
X_test_raw = data_raw(root_path, X_test_path)
y_train = data_raw(root_path, y_train_path)

In [5]:
X_train_raw = median_imp(X_train_raw)
X_test_raw = median_imp(X_test_raw)

In [6]:
mi_selector = feat_sel(X_train_raw, y_train, mutual_info_regression, num_features)

X_train_raw = mi_selector.transform(X_train_raw)
X_test = mi_selector.transform(X_test_raw)

In [7]:
X_train_raw, X_val, y_train, y_val = train_val_split(X_train_raw, y_train, val_size, random_state)

In [8]:
X_train, y_train = out_det(X_train_raw, y_train, random_state)

In [9]:
X_train = StandardScaler().fit_transform(X_train)
X_val = StandardScaler().fit_transform(X_val)
X_test = StandardScaler().fit_transform(X_test)

In [10]:
svr = SVR().fit(X_train, y_train.ravel())

In [11]:
y_val_pred = svr.predict(X_val)

In [12]:
score = r2_score(y_val, y_val_pred)
print(score)

0.5019024311202779


In [13]:
y_test_pred = svr.predict(X_test)

In [15]:
df_result = pd.DataFrame(data = y_test_pred, columns=['y'])
df_result.to_csv(path_or_buf=os.path.join(root_path,y_test_path), index_label='id')